
# Competitor Price & Availability Monitoring with Scrapy
## Problem Statement (Business Context)
A retail analytics company needs to **monitor competitors’ product pricing and availability** across multiple e‑commerce websites.  
Manually checking prices daily is **inefficient, error-prone, and slow**. We need an **automated crawler** that:

- Visits category and product pages on competitor sites
- Extracts **product title, price, availability, rating, and seller information**
- Handles **pagination** to cover full catalogs
- **Cleans & stores** results (CSV/JSON/SQLite) for BI dashboards
- Can be **scheduled** to run daily/weekly for trend analysis

### Success Criteria
- Reliable, repeatable crawls with throttling and retries
- Clean, structured data exports
- Modular code (spiders, items, pipelines, settings) ready for production

---

> **Note**: In this notebook we use the public demo site **Books to Scrape** (`https://books.toscrape.com/`) as a stand‑in for real e‑commerce sites.  
> Swap CSS/XPath selectors when adapting to your actual target sites, and follow each site's Terms of Service and robots.txt.



## Architecture Overview
**Scrapy Project Structure**
```
competitor_monitor/
├─ competitor_monitor/
│  ├─ __init__.py
│  ├─ items.py          # Data model (Item fields)
│  ├─ middlewares.py    # Custom request/response middleware (optional)
│  ├─ pipelines.py      # Clean/validate/enrich & store
│  ├─ settings.py       # Throttling, retries, user-agent, etc.
│  └─ spiders/
│     └─ books_spider.py  # Example spider (replace with your target)
├─ scrapy.cfg
```
**Data Flow**
`Request` → `Response` → **Spider.parse** → `Item` → **Pipelines** → Storage (CSV/JSON/SQLite)

**Key Techniques**
- **Pagination**: follow "next" links
- **Robustness**: retries, backoff, throttling, caching
- **Data Quality**: schema via `Item`, cleanup in pipelines
- **Scheduling**: cron / Airflow / CI



## 1) Environment Setup
Run the following cell once to install dependencies (locally or in Colab). Skip if Scrapy is already installed.


In [ ]:

# If running in a fresh environment, uncomment and run:
# !pip install scrapy==2.11.2 parsel==1.9.1 itemadapter==0.9.0



**Why this?**  
We fix versions to ensure reproducibility. `scrapy` is the core framework; `parsel` provides CSS/XPath parsing; `itemadapter` helps Items/DataClass conversions in pipelines.



## 2) Quick Start: Run a Spider Programmatically (No `scrapy startproject` needed)
For learning and demo, we'll run Scrapy **entirely from this notebook** using `CrawlerProcess`.  
In production you’d usually create a project with `scrapy startproject` and run `scrapy crawl <name>`.


In [1]:

from pathlib import Path
from datetime import datetime

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_CSV = DATA_DIR / f"books_export_{timestamp}.csv"

OUTPUT_CSV


PosixPath('data/books_export_20250820_150828.csv')


**Why this?**  
We prepare a **timestamped output path** so every run produces a separate file, simplifying lineage and audits.



## 3) Define a Structured Item
An `Item` standardizes the fields we expect. This improves validation and downstream analytics.


In [3]:
pip install scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.7/264.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.2 MB/s eta 0:00:00


In [4]:

import scrapy

class ProductItem(scrapy.Item):
    product_id = scrapy.Field()     # stable identifier if available
    title = scrapy.Field()
    price = scrapy.Field()
    availability = scrapy.Field()
    rating = scrapy.Field()
    product_url = scrapy.Field()
    category = scrapy.Field()
    crawl_ts = scrapy.Field()


## Understaning :
scrapy.Item → like a data container (similar to a dictionary but with structure).

Each scrapy.Field() → defines a field you want to capture from the website.

ProductItem → ensures scraped data is stored in a clean, consistent format (easy for exporting to CSV/JSON or databases).

👉 In short: This class defines the schema for scraped product data (ID, title, price, stock, rating, link, category, timestamp).


**Why this?**  
Defining a **schema** up front forces consistent outputs and makes pipeline validation easier.



## 4) Build the Spider (Pagination + Extraction)
We'll crawl **Books to Scrape** categories and extract product rows from listing pages, then follow pagination.


In [5]:

import re
import scrapy
from urllib.parse import urljoin
from datetime import datetime

class BooksSpider(scrapy.Spider):
    name = "books_demo"
    allowed_domains = ["books.toscrape.com"]
    start_urls = ["https://books.toscrape.com/catalogue/category/books_1/index.html"]

    custom_settings = {
        "DOWNLOAD_DELAY": 0.5,
        "RANDOMIZE_DOWNLOAD_DELAY": True,
        "CONCURRENT_REQUESTS": 8,
        "RETRY_ENABLED": True,
        "RETRY_TIMES": 2,
        "AUTOTHROTTLE_ENABLED": True,
        "AUTOTHROTTLE_START_DELAY": 0.5,
        "AUTOTHROTTLE_MAX_DELAY": 5,
        "AUTOTHROTTLE_TARGET_CONCURRENCY": 1.0,
        "FEEDS": {},
        "HTTPCACHE_ENABLED": False
    }

    def parse(self, response):
        category = response.css("div.page-header.action h1::text").get()
        if not category:
            category = response.css("ul.breadcrumb li.active::text").get()

        for prod in response.css("section div ol.row li"):
            title = prod.css("h3 a::attr(title)").get()
            url = urljoin(response.url, prod.css("h3 a::attr(href)").get())
            price_text = prod.css("div.product_price p.price_color::text").get()
            availability_text = prod.css("div.product_price p.instock.availability::text").getall()
            availability_text = " ".join([t.strip() for t in availability_text if t.strip()])

            rating_class = prod.css("p.star-rating::attr(class)").get() or ""
            m = re.search(r"star-rating\s+(\w+)", rating_class)
            rating = m.group(1) if m else None

            price = None
            if price_text:
                m = re.search(r"([0-9]+\.[0-9]+)", price_text)
                price = float(m.group(1)) if m else None

            item = ProductItem(
                product_id=None,
                title=title,
                price=price,
                availability=availability_text,
                rating=rating,
                product_url=url,
                category=category,
                crawl_ts=datetime.utcnow().isoformat()
            )
            yield item

        next_href = response.css("li.next a::attr(href)").get()
        if next_href:
            yield response.follow(next_href, callback=self.parse)



**Why this?**  
- **`allowed_domains` / `start_urls`**: constrain the crawl and establish entry points.  
- **Politeness**: `DOWNLOAD_DELAY`, `AUTOTHROTTLE` avoid hammering servers.  
- **Extraction**: CSS selectors parse list cards; regex normalizes price; rating derived from class names.  
- **Pagination**: we follow the "Next" link to cover the full category.  
- **Timestamps**: `crawl_ts` enables time‑series tracking for pricing trends.


Understaning : This code is a **Scrapy spider** that crawls the demo site [books.toscrape.com](https://books.toscrape.com) and extracts book details. Let’s break it down concisely:

---

### 1. **Imports**

```python
import re, scrapy
from urllib.parse import urljoin
from datetime import datetime
```

* `re`: for regex (extracting numbers/rating).
* `scrapy`: the web-scraping framework.
* `urljoin`: builds absolute URLs from relative ones.
* `datetime`: adds a timestamp for when data was scraped.

---

### 2. **Spider Definition**

```python
class BooksSpider(scrapy.Spider):
    name = "books_demo"
    allowed_domains = ["books.toscrape.com"]
    start_urls = ["https://books.toscrape.com/catalogue/category/books_1/index.html"]
```

* Defines a spider named `books_demo`.
* Restricts crawling to `books.toscrape.com`.
* Starts from the “Books” category page.

---

### 3. **Custom Settings**

```python
custom_settings = {
    "DOWNLOAD_DELAY": 0.5,               # wait between requests
    "RANDOMIZE_DOWNLOAD_DELAY": True,    # add randomness
    "CONCURRENT_REQUESTS": 8,            # max simultaneous requests
    "RETRY_ENABLED": True, "RETRY_TIMES": 2,   # retry on errors
    "AUTOTHROTTLE_ENABLED": True,        # auto slow down if server stressed
    "AUTOTHROTTLE_START_DELAY": 0.5,
    "AUTOTHROTTLE_MAX_DELAY": 5,
    "AUTOTHROTTLE_TARGET_CONCURRENCY": 1.0,
    "FEEDS": {},                         # export feeds (empty here)
    "HTTPCACHE_ENABLED": False           # caching disabled
}
```

👉 Controls crawl behavior for politeness and performance.

---

### 4. **Main Parser**

```python
def parse(self, response):
    category = response.css("div.page-header.action h1::text").get()
    if not category:
        category = response.css("ul.breadcrumb li.active::text").get()
```

* Gets category name (from header or breadcrumb).

---

### 5. **Extract Each Book**

```python
for prod in response.css("section div ol.row li"):
    title = prod.css("h3 a::attr(title)").get()
    url = urljoin(response.url, prod.css("h3 a::attr(href)").get())
    price_text = prod.css("div.product_price p.price_color::text").get()
    availability_text = prod.css("div.product_price p.instock.availability::text").getall()
    availability_text = " ".join([t.strip() for t in availability_text if t.strip()])
```

* Extracts book **title**, **URL**, **price text**, and **availability text**.

---

### 6. **Get Rating**

```python
rating_class = prod.css("p.star-rating::attr(class)").get() or ""
m = re.search(r"star-rating\s+(\w+)", rating_class)
rating = m.group(1) if m else None
```

* Star rating is stored as a CSS class like `"star-rating Three"`.
* Regex extracts the rating word (e.g., `"Three"`).

---

### 7. **Parse Price**

```python
price = None
if price_text:
    m = re.search(r"([0-9]+\.[0-9]+)", price_text)
    price = float(m.group(1)) if m else None
```

* Extracts numeric price (`£51.77` → `51.77` as `float`).

---

### 8. **Build Item**

```python
item = ProductItem(
    product_id=None,
    title=title,
    price=price,
    availability=availability_text,
    rating=rating,
    product_url=url,
    category=category,
    crawl_ts=datetime.utcnow().isoformat()
)
yield item
```

* Creates a structured `ProductItem` (from your earlier code).
* Adds `crawl_ts` = timestamp.
* `yield` sends it to pipeline/export.

---

### 9. **Pagination Handling**

```python
next_href = response.css("li.next a::attr(href)").get()
if next_href:
    yield response.follow(next_href, callback=self.parse)
```

* Looks for a “Next” button.
* If found, follows the link and continues parsing the next page.

---

✅ **In Short:**
This spider scrapes **book title, URL, price, availability, rating, category, and crawl timestamp** from *books.toscrape.com*, handles multiple pages via pagination, and applies crawl throttling to be polite.

---



## 5) Pipelines (Clean → Store)
We’ll do simple normalization and write to both **CSV** and **SQLite** to show two common sinks.


In [6]:

from itemadapter import ItemAdapter
import sqlite3

class CleanAndStorePipeline:
    def open_spider(self, spider):
        # Prepare SQLite
        self.conn = sqlite3.connect(str(DATA_DIR / "products.sqlite"))
        self.cur = self.conn.cursor()
        self.cur.execute("""
            CREATE TABLE IF NOT EXISTS products (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                product_id TEXT,
                title TEXT,
                price REAL,
                availability TEXT,
                rating TEXT,
                product_url TEXT,
                category TEXT,
                crawl_ts TEXT
            )
        """)
        self.conn.commit()

    def process_item(self, item, spider):
        ad = ItemAdapter(item)

        title = (ad.get("title") or "").strip()
        availability = (ad.get("availability") or "").strip()
        rating = (ad.get("rating") or "").strip() if ad.get("rating") else None

        self.cur.execute("""
            INSERT INTO products (product_id, title, price, availability, rating, product_url, category, crawl_ts)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            ad.get("product_id"),
            title,
            ad.get("price"),
            availability,
            rating,
            ad.get("product_url"),
            ad.get("category"),
            ad.get("crawl_ts"),
        ))
        self.conn.commit()

        ad["title"] = title
        ad["availability"] = availability
        ad["rating"] = rating
        return ad.item

    def close_spider(self, spider):
        self.conn.close()



**Why this?**  
- Centralizes **cleanup** (strip whitespace, normalize fields).  
- Demonstrates **SQLite** storage for BI queries while also allowing CSV export via `FEEDS`.  
- Keeps **I/O in pipelines** to keep spiders focused on crawling/extraction.


## Understanding
This code defines a **Scrapy pipeline** that **cleans scraped data** and **stores it in an SQLite database**. Let’s break it down:

---

### 1. **Imports**

```python
from itemadapter import ItemAdapter
import sqlite3
```

* `ItemAdapter`: safely access fields from Scrapy `Item` or dict-like objects.
* `sqlite3`: Python’s built-in library for working with SQLite databases.

---

### 2. **Pipeline Class**

```python
class CleanAndStorePipeline:
```

* A Scrapy pipeline processes items after they’re scraped (e.g., cleaning, validation, storage).

---

### 3. **Open Database at Spider Start**

```python
def open_spider(self, spider):
    self.conn = sqlite3.connect(str(DATA_DIR / "products.sqlite"))
    self.cur = self.conn.cursor()
    self.cur.execute("""
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            product_id TEXT,
            title TEXT,
            price REAL,
            availability TEXT,
            rating TEXT,
            product_url TEXT,
            category TEXT,
            crawl_ts TEXT
        )
    """)
    self.conn.commit()
```

* Runs **once when spider starts**.
* Connects to `products.sqlite` in `DATA_DIR`.
* Creates a `products` table if it doesn’t already exist.
* Schema matches your `ProductItem`.

---

### 4. **Process Each Item**

```python
def process_item(self, item, spider):
    ad = ItemAdapter(item)

    # Clean fields
    title = (ad.get("title") or "").strip()
    availability = (ad.get("availability") or "").strip()
    rating = (ad.get("rating") or "").strip() if ad.get("rating") else None
```

* `ItemAdapter` makes item access consistent.
* Cleans text fields (`strip()` removes extra spaces).
* Handles `None` safely for missing fields.

```python
    # Insert into DB
    self.cur.execute("""
        INSERT INTO products (product_id, title, price, availability, rating, product_url, category, crawl_ts)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        ad.get("product_id"),
        title,
        ad.get("price"),
        availability,
        rating,
        ad.get("product_url"),
        ad.get("category"),
        ad.get("crawl_ts"),
    ))
    self.conn.commit()
```

* Inserts the cleaned data into SQLite.
* Uses parameterized query `?` to prevent SQL injection.
* Commits after each insert.

```python
    # Update cleaned fields back into the item
    ad["title"] = title
    ad["availability"] = availability
    ad["rating"] = rating
    return ad.item
```

* Updates the item with cleaned values.
* Returns it for further processing/export.

---

### 5. **Close Database at Spider End**

```python
def close_spider(self, spider):
    self.conn.close()
```

* Closes the DB connection when spider finishes.

---

✅ **In Short:**
This pipeline takes each scraped product, **cleans text fields**, and **stores it in an SQLite database (`products.sqlite`)** with a well-defined schema.

---




## 6) Settings for This Notebook Run
We wire the pipeline and feed exports via `CrawlerProcess` settings.


In [7]:

from scrapy.crawler import CrawlerProcess

process_settings = {
    "USER_AGENT": "CompetitorMonitorBot/1.0 (+https://example.com/bot-info)",
    "ROBOTSTXT_OBEY": False,  # True in real projects when appropriate
    "DOWNLOAD_DELAY": 0.5,
    "RANDOMIZE_DOWNLOAD_DELAY": True,
    "CONCURRENT_REQUESTS": 8,
    "RETRY_TIMES": 2,
    "AUTOTHROTTLE_ENABLED": True,
    "AUTOTHROTTLE_START_DELAY": 0.5,
    "AUTOTHROTTLE_MAX_DELAY": 5,
    "AUTOTHROTTLE_TARGET_CONCURRENCY": 1.0,
    "ITEM_PIPELINES": {
        "__main__.CleanAndStorePipeline": 300,
    },
    "FEEDS": {
        str(OUTPUT_CSV): {"format": "csv", "overwrite": True},
    },
    "HTTPCACHE_ENABLED": False,
    "DOWNLOAD_TIMEOUT": 30,
    "LOG_LEVEL": "INFO",
}

process_settings


{'USER_AGENT': 'CompetitorMonitorBot/1.0 (+https://example.com/bot-info)',
 'ROBOTSTXT_OBEY': False,
 'DOWNLOAD_DELAY': 0.5,
 'RANDOMIZE_DOWNLOAD_DELAY': True,
 'CONCURRENT_REQUESTS': 8,
 'RETRY_TIMES': 2,
 'AUTOTHROTTLE_ENABLED': True,
 'AUTOTHROTTLE_START_DELAY': 0.5,
 'AUTOTHROTTLE_MAX_DELAY': 5,
 'AUTOTHROTTLE_TARGET_CONCURRENCY': 1.0,
 'ITEM_PIPELINES': {'__main__.CleanAndStorePipeline': 300},
 'FEEDS': {'data/books_export_20250820_150828.csv': {'format': 'csv',
   'overwrite': True}},
 'HTTPCACHE_ENABLED': False,
 'DOWNLOAD_TIMEOUT': 30,
 'LOG_LEVEL': 'INFO'}


**Why this?**  
- **`USER_AGENT`**: identify your bot.  
- **`ROBOTSTXT_OBEY`**: consider enabling in production if you must respect robots rules.  
- **`ITEM_PIPELINES`**: activates our cleanup & storage step.  
- **`FEEDS`**: exports a CSV for analysts/dashboards.  
- **`AUTOTHROTTLE` / `RETRY`**: production hygiene to reduce failures and server load.


## Understanding
This code defines a **Scrapy crawler configuration dictionary** (`process_settings`) that controls how the spider runs when launched programmatically (with `CrawlerProcess`).

Let’s break it down:

---

### 1. **Crawler Setup**

```python
from scrapy.crawler import CrawlerProcess
```

* `CrawlerProcess` allows running Scrapy spiders **inside a Python script** (instead of only with `scrapy crawl` from terminal).

---

### 2. **Settings Dictionary**

```python
process_settings = {
    "USER_AGENT": "CompetitorMonitorBot/1.0 (+https://example.com/bot-info)",
```

* Sets the **User-Agent** header (identifies your bot).

```python
    "ROBOTSTXT_OBEY": False,  # True in real projects when appropriate
```

* If `True`, Scrapy respects `robots.txt` rules.
* Here it’s `False` (will crawl even if disallowed — fine for learning/demo but not ethical in production).

---

### 3. **Politeness & Performance**

```python
    "DOWNLOAD_DELAY": 0.5,              # half-second delay between requests
    "RANDOMIZE_DOWNLOAD_DELAY": True,   # adds randomness to avoid detection
    "CONCURRENT_REQUESTS": 8,           # max requests at once
```

* Prevents hammering the server.

```python
    "RETRY_TIMES": 2,                   # retry failed requests twice
```

---

### 4. **AutoThrottle**

```python
    "AUTOTHROTTLE_ENABLED": True,
    "AUTOTHROTTLE_START_DELAY": 0.5,
    "AUTOTHROTTLE_MAX_DELAY": 5,
    "AUTOTHROTTLE_TARGET_CONCURRENCY": 1.0,
```

* Dynamically adjusts crawl speed based on server load.
* Starts at 0.5s delay, can back off up to 5s.
* Keeps \~1 request in flight at a time.

---

### 5. **Pipelines**

```python
    "ITEM_PIPELINES": {
        "__main__.CleanAndStorePipeline": 300,
    },
```

* Enables the pipeline `CleanAndStorePipeline` (the one storing data into SQLite).
* Priority `300` determines execution order (lower number = higher priority).

---

### 6. **Feed Exports**

```python
    "FEEDS": {
        str(OUTPUT_CSV): {"format": "csv", "overwrite": True},
    },
```

* Saves scraped data into a CSV file (`OUTPUT_CSV`).
* `overwrite=True` ensures each run replaces the file instead of appending.

---

### 7. **Other Controls**

```python
    "HTTPCACHE_ENABLED": False,   # disables caching
    "DOWNLOAD_TIMEOUT": 30,       # request timeout in seconds
    "LOG_LEVEL": "INFO",          # logging verbosity
}
```

* No HTTP cache (fresh requests each time).
* Requests that take longer than 30s are dropped.
* Logs only `INFO` and above (no debug noise).

---

✅ **In Short:**
This dictionary defines how the Scrapy spider behaves:

* Identifies itself with a custom User-Agent.
* Controls request rate, retries, and throttling.
* Sends items through your SQLite pipeline.
* Also writes items to a CSV file.
* Runs with polite timeouts and moderate logging.

---




## 7) (Optional) Execute the Crawl from the Notebook
> This will attempt to access the live demo site. If your environment blocks outbound internet,
> keep this as reference and run locally instead.

Running this cell will:
1. Spin up Scrapy
2. Crawl all "Books" pages
3. Write cleaned rows to **SQLite** and **CSV**


In [ ]:

# NOTE: This may not run in restricted environments with no internet access.
# Uncomment to run where outbound HTTP is allowed.
#
# process = CrawlerProcess(settings=process_settings)
# process.crawl(BooksSpider)
# process.start()
#
# print(f"CSV export -> {OUTPUT_CSV}")
# print(f"SQLite DB  -> {DATA_DIR / 'products.sqlite'}")



**Why this?**  
Shows how to **programmatically** run Scrapy from Python—handy for automation/orchestration (Airflow/Cron) and unit/integration tests.



## 8) Data Validation & Quick QA
After a run, validate schema and count rows. In real life, add unit tests and anomaly checks (e.g., price spikes/drops).


In [8]:

import sqlite3, pandas as pd

db_path = DATA_DIR / "products.sqlite"
if db_path.exists():
    conn = sqlite3.connect(str(db_path))
    df = pd.read_sql_query("SELECT title, price, availability, rating, category, crawl_ts FROM products LIMIT 10;", conn)
    conn.close()
    df.head()
else:
    print("Run the crawl first to populate the SQLite database.")


Run the crawl first to populate the SQLite database.



**Why this?**  
Sanity‑checks the **shape and quality** of the data before sending to BI tools. You can expand this with checks for nulls, outliers, and duplicates.



## 9) Scaling to Multiple Sites (Pattern)
- Create one spider **per site** (`amazon_spider.py`, `flipkart_spider.py`, …).  
- Reuse shared pipelines (cleanup, SQLite/warehouse writers).  
- Add **site‑specific** parsers/selectors; normalize fields to the same `Item` schema.  
- Use **rotating proxies** and **headless browsers** (e.g., Splash/Playwright) for JavaScript‑heavy sites where allowed.  
- Centralize scheduling (cron/Airflow/GitHub Actions).



## 10) Scheduling
**Cron example (daily at 6 AM):**
```
0 6 * * * /usr/bin/env python /path/to/run_spiders.py >> /var/log/price_crawl.log 2>&1
```

**Airflow**: create a DAG that runs a PythonOperator calling `CrawlerProcess` with your spiders.



## 11) Compliance, Ethics, and Legal Notes
- Respect each site’s **Terms of Service** and **robots.txt**.
- Avoid PII collection. Store only what you need.
- Rate‑limit politely; add contact info in your `User-Agent` string.



## 12) Next Steps
- Add more fields (brand, seller, shipping, discount, MRP).  
- Implement **delta‑detection** to store only changes (smaller DB, easier diffing).  
- Build a **dashboard** (Power BI / Tableau / Streamlit) for price trends and stockouts.
